In [1]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
from utils import *
from distances import *
from consensus import *
import math
import sklearn
import os
from pathlib import Path
import json
from subprocess import Popen, PIPE
import subprocess
import time
import sys

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'karateclub', 'graph_tool'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer'}


In [2]:
cons_name = "refined-bilouvain-merge"

In [97]:
def bilouvain_merge(P_star_refined, P_list):
    refined_partition = list(P_star_refined["partition"])
    supernode_names = []
    for i in range(len(refined_partition)):
        supernode_names.append("super_"+str(i))
    
    cluster_names = []
    for k in range(len(P_list)):
        for j in range(len(P_list[k]["partition"])):
            cluster_names.append("cluster_" + str(k) + "_" + str(j))
    
    #print(cluster_names)
    #print(supernode_names)
    
    adj_original = nx.to_numpy_array(P_star_refined["graph"])
    
    n_supernodes = len(supernode_names)
    adj_coarsened = np.zeros((n_supernodes, n_supernodes))
    for i in range(n_supernodes):
        for j in range(n_supernodes):
            for ei in refined_partition[i]:
                for ej in refined_partition[j]:
                    x = int(ei)
                    y = int(ej)
                    adj_coarsened[i,j] = adj_coarsened[i,j] + adj_original[x,y]
                    adj_coarsened[j,i] = adj_coarsened[j,i] + adj_original[y,x]
    
    #print(adj_coarsened)
    #print(adj_coarsened.shape)
    #print(np.count_nonzero(adj_coarsened))
    
    n_clusters = len(cluster_names)
    n = n_supernodes + n_clusters # Total number of vertices in bipartite graph
    
    #print(n_supernodes)
    #print(n_clusters)
    #print(n)
    
    adj_bipartite = np.zeros((n,n))
    si = 0 # Supernode index starts from 0
    ci = n_supernodes # Cluster node index starts from n_supernodes
    
    for k in range(len(P_list)):
        #print(k, "/", len(P_list))
        for j in range(len(P_list[k]["partition"])):
            #print(j, "-", len(P_list[k]["partition"]))
            for si in range(n_supernodes):
                #print(si, "/", n_supernodes, " == ", si, ",", ci)
                if(set(refined_partition[si]) <= set(P_list[k]["partition"][j])):
                    adj_bipartite[si,ci] = len(refined_partition[si])
                    adj_bipartite[ci,si] = len(refined_partition[si])
            ci = ci + 1
                    
    #print(adj_bipartite)
    #print(adj_bipartite.shape)
    #print(np.count_nonzero(adj_bipartite))
    
    Gb = nx.from_numpy_matrix(adj_bipartite)
    nx.write_weighted_edgelist(Gb, "biLouvain.bipartite", delimiter="\t")
    
    cmdlist = [
                "/nfs/nfs2/home/mth/Codes/biLouvain/src/biLouvain",
                "-i",
                "biLouvain.bipartite",
                "-d",
                "\\t",
                #"-o",
                #"biLouvain_output",
    ]
    proc = subprocess.Popen(cmdlist, stdout=PIPE)
    #output, err = proc.communicate()
    proc.wait()
    
    # Parse biLouvain output and extract necessary information
    biLouvain_cluster_file = "biLouvain_ResultsCommunities.txt" 
    biLouvain_clust_lst = []
    with open(biLouvain_cluster_file, mode="r") as infile:
        lines = infile.readlines()
        for l in lines:
            if (len(l) >= 9) and (l[0:9] == "Community") and ("V1" in l):
                lst = []
                linetoks = l.split(":")
                #print(linetoks)
                relevant_section = linetoks[1].strip()
                clusttoks = relevant_section.split(",")
                for item in clusttoks:
                    lst.append(item)
                biLouvain_clust_lst.append(list(lst))
    
    # Remove files created to run biLouvain
    this_dir = "."
    for fname in os.listdir(this_dir):
        if fname.startswith("biLouvain"):
            os.remove(os.path.join(this_dir, fname))
    
    # Merge refined partitions depending on biLouvain output
    merged_partition = []
    for bcl in biLouvain_clust_lst:
        merged_cluster = []
        for e in bcl:
            i = int(e)
            merged_cluster = merged_cluster + refined_partition[i]
        merged_partition.append(merged_cluster)
    
    print(merged_partition)
    P_star = { "graph": nx.Graph(P_star_refined["graph"]), "partition": list(merged_partition)}
    return P_star

In [99]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy import sparse
import pandas as pd

algs = ["infomap", "louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
#mus = [1, 2, 3, 4, 5, 6, 7, 8, 9]
mus = [1]
gammas = [30]
betas = [11]

all_stats = []

for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_matrix(spio.mmread(f), create_using=nx.Graph)
                for alg in algs:
                    clust_file = fileprefix + fname + "." + alg
                    partition = read_clust_lst(clust_file)
                    P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                #P_star_refined = refined_consensus(P_list)
                
                refined_partition = read_clust_lst(fileprefix + fname + "." + "refined")
                P_star_refined = {"graph": nx.Graph(G), "partition": list(refined_partition)}
                
                P_star_refined_bilouvain_merge = bilouvain_merge(P_star_refined, P_list)
                
                #print(P_star_refined_bilouvain_merge)
                #write_clust_lst(P_star_refined_bilouvain_merge["partition"], fileprefix + fname + "." + cons_name)
                print(cons_name, len(P_star_refined_bilouvain_merge["partition"]))

LFR/n1000/LFR_n1000_mu01_gamma30_beta11.mtx


FileNotFoundError: [Errno 2] No such file or directory: 'biLouvain_ResultsCommunities.txt'